In [ ]:
import os

import mlflow


EXPERIMENT_NAME = "churn_marselkamilov"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_marselkamilov"


TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('S3_ACCESS_KEY')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('S3_SECRET_KEY')


mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")


pip_requirements = '../requirements.txt' # ваш код здесь
signature = mlflow.models.infer_signature(X_test, prediction) # ваш код здесь
input_example = X_test[:10] # ваш код здесь
metadata = {'model_type': 'monthly'} # ваш код здесь


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metrics(metrics)
        
    model_info = mlflow.catboost.log_model(cb_model=model,
        artifact_path="models",
        signature=signature,
        metadata=metadata,
        input_example=input_example,
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        await_registration_for=60
		)

In [ ]:
client = mlflow.MlflowClient() 

REGISTRY_MODEL_NAME = "churn_model_marselkamilov"

models = client.search_model_versions(
    filter_string=f"name = '{REGISTRY_MODEL_NAME}'"
)
print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage


print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

# поменяйте статус каждой модели
client.transition_model_version_stage(model_name_1, model_version_1, "production")
client.transition_model_version_stage(model_name_2, model_version_2, "staging") 

# переимнуйте модель в реестре
client.rename_registered_model(
    name=REGISTRY_MODEL_NAME, 
    new_name=f"{REGISTRY_MODEL_NAME}_b2c"
) 

In [ ]:
# Доступ через MLflow Python API
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

client = mlflow.MlflowClient(tracking_uri=tracking_uri, registry_uri=registry_uri)

EXPERIMENT_NAME = "experiment_name"

experiment_id = client.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

# Запуски внутри эксперимента
experiment_runs = mlflow.search_runs(experiment_ids=[experiment_id],).sort_values(by="start_time", ascending=False) 

# Метрики
runs = experiment_runs[["run_id", "start_time",
                        "metrics.err1","metrics.err2",
                        "metrics.logloss","metrics.recall",
                        "metrics.auc","metrics.f1","metrics.precision"]]

# фиксируем интересующий запуск (run_id) и метрику
run_id = "d9f8d580f14040408e8d4645e2bcaaf2"
metric = "logloss"

print(client.get_metric_history(run_id, metric)) 

client.get_run(run_id).data.metrics

# Артефакты
run = client.get_run(run)
artifact_uri = run.info.artifact_uri

print(f"Путь артефакта в S3: '{artifact_uri}'") 

mlflow.artifacts.download_artifacts(artifact_uri, dst_path="./artifacts_local")

REGISTRY_MODEL_NAME = "YP_MODEL_new"
REGISTRY_MODEL_VERSION = "1"

model_uri = client.get_model_version_download_uri(
    REGISTRY_MODEL_NAME,
    REGISTRY_MODEL_VERSION,
)

model = mlflow.catboost.load_model(model_uri)

print(f"Путь до модели {REGISTRY_MODEL_NAME} версии {REGISTRY_MODEL_VERSION} в S3: '{model_uri}'")

model = mlflow.xgboost.load_model(model_uri)

# Хранилища признаков и артефактов
